In [82]:
import re
import os
import time
from pytube import YouTube
import pandas as pd
from tqdm import tqdm

In [83]:

data = pd.read_csv('/Users/amirdjanibekov/Documents/MBZUAI/NLP703/nlp703-speech-processing/csv/all_music_genre_wo_text.csv', encoding='UTF-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5523 entries, 0 to 5522
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5523 non-null   object
 1   ondblclick   5523 non-null   object
 2   onmouseover  5523 non-null   object
 3   class        5523 non-null   object
dtypes: object(4)
memory usage: 172.7+ KB


In [84]:
data.iloc[189]

id                                                       item190
ondblclick     window.open("https://www.google.com/search?q=A...
onmouseover    playx("494.m4a.mp3", "190", "38% Awe-inspiring...
class                                                          F
Name: 189, dtype: object

In [85]:
data.head()

,id,ondblclick,onmouseover,class
0,item1,"window.open(""https://www.amazon.com/dp/B00B6N6...","playx(""01 - Tree Dance&1.mp3"", ""1"", ""57% Calm,...",E
1,item2,"window.open(""https://www.youtube.com/watch?v=0...","playx(""04F4xlWSFh0_18.mp3"", ""2"", ""42% Energizi...",G
2,item3,"window.open(""https://www.youtube.com/watch?v=0...","playx(""04F4xlWSFh0_75.mp3"", ""3"", ""50% Energizi...",G
3,item4,"window.open(""https://www.youtube.com/watch?v=0...","playx(""05UApr2UQHQ_227.mp3"", ""4"", ""20% Calm, r...",F
4,item5,"window.open(""https://www.youtube.com/watch?v=0...","playx(""073TNjLc070_40.mp3"", ""5"", ""29% Energizi...",G


In [86]:
data['link'] = data['ondblclick'].apply(
    lambda x: (
            re.search(r'(?P<link>https.*)",',x) or
            re.search(r'(?P<link>.*)', x)
    ).group('link')
)
data['domain'] = data['ondblclick'].apply(
    lambda x: (
            re.search(r'www\.(?P<domain>.*)\.\w',x) or
            re.search(r'(?P<domain>.*)', x)
    ).group('domain')
)

In [87]:
data['domain'].value_counts(normalize=True)

domain
youtube                                                                         0.884845
google                                                                          0.054861
woodzie                                                                         0.050516
wagnerheim                                                                      0.007605
window.open("http://music.sailingconductors.com/track/nocturne","mywindow");    0.001630
amazon                                                                          0.000543
Name: proportion, dtype: float64

In [88]:
youtube = data[data['domain'] == 'youtube']
youtube

,id,ondblclick,onmouseover,class,link,domain
1,item2,"window.open(""https://www.youtube.com/watch?v=0...","playx(""04F4xlWSFh0_18.mp3"", ""2"", ""42% Energizi...",G,https://www.youtube.com/watch?v=04F4xlWSFh0,youtube
2,item3,"window.open(""https://www.youtube.com/watch?v=0...","playx(""04F4xlWSFh0_75.mp3"", ""3"", ""50% Energizi...",G,https://www.youtube.com/watch?v=04F4xlWSFh0,youtube
3,item4,"window.open(""https://www.youtube.com/watch?v=0...","playx(""05UApr2UQHQ_227.mp3"", ""4"", ""20% Calm, r...",F,https://www.youtube.com/watch?v=05UApr2UQHQ,youtube
4,item5,"window.open(""https://www.youtube.com/watch?v=0...","playx(""073TNjLc070_40.mp3"", ""5"", ""29% Energizi...",G,https://www.youtube.com/watch?v=073TNjLc070,youtube
5,item6,"window.open(""https://www.youtube.com/watch?v=0...","playx(""0DugzooGWKk_1.mp3"", ""6"", ""31% Calm, rel...",J,https://www.youtube.com/watch?v=0DugzooGWKk,youtube
...,...,...,...,...,...,...
5518,item1837,"window.open(""https://www.youtube.com/watch?v=z...","playx(""zhRzORqNa0E_5.mp3"", ""1837"", ""34% Beauti...",E,https://www.youtube.com/watch?v=zhRzORqNa0E,youtube
5519,item1838,"window.open(""https://www.youtube.com/watch?v=z...","playx(""zk93KbUtFyA_6.mp3"", ""1838"", ""50% Energi...",G,https://www.youtube.com/watch?v=zk93KbUtFyA,youtube
5520,item1839,"window.open(""https://www.youtube.com/watch?v=z...","playx(""zn9vVjg0kZU_1027.mp3"", ""1839"", ""41% Cal...",F,https://www.youtube.com/watch?v=zn9vVjg0kZU,youtube
5521,item1840,"window.open(""https://www.youtube.com/watch?v=z...","playx(""zn9vVjg0kZU_13.mp3"", ""1840"", ""50% Calm,...",E,https://www.youtube.com/watch?v=zn9vVjg0kZU,youtube


In [99]:
def download_from_youtube(row):
    # time.sleep(5)
    try:
        yt = YouTube(row['link'])
        video = yt.streams.filter(only_audio=True).first()

        destination = 'music-full/'
        out_file = video.download(output_path=destination)

        os.rename(out_file, destination + row["id"] + '.mp3')
    except KeyError:
        print('error')
        with open('errors_while_downloading.txt', 'a') as file:
            file.write(row['id'] + ',' + row['link'])
    except Exception:
        with open('error.txt', 'a') as file:
            file.write(row['id'] + ', ' + row['link'])

In [100]:
tqdm.pandas()
# instantiate
youtube[:2443].progress_apply(download_from_youtube, axis=1)

  0%|          | 5/2443 [00:17<2:31:12,  3.72s/it]

error


  0%|          | 8/2443 [00:30<2:37:24,  3.88s/it]

error


  0%|          | 9/2443 [00:33<2:26:16,  3.61s/it]

...

error


  1%|          | 25/2443 [01:40<2:06:19,  3.13s/it]

error


  1%|          | 27/2443 [01:52<2:48:19,  4.18s/it]


KeyboardInterrupt: 